In [ ]:
# installing the Kaggle library
!pip install kaggle

In [ ]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# API to fetch the dataset from Kaggle
!kaggle competitions download -c twitter-sentiment-analysis2

 46% 5.00M/11.0M [00:00<00:00, 50.7MB/s]
100% 11.0M/11.0M [00:00<00:00, 53.1MB/s]
  0% 0.00/3.67M [00:00<?, ?B/s]
100% 3.67M/3.67M [00:00<00:00, 197MB/s]


In [ ]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset_train = 'file_path'
dataset_test = 'file_path'

In [16]:
with ZipFile(dataset_train,'r') as zip:
  zip.extractall()
with ZipFile(dataset_test,'r') as zip1:
  zip1.extractall()
  

In [204]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [205]:
#nltk.download('punkt')

In [206]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [207]:
# printing the stopwords in English
stop_words=set(stopwords.words('english'))
print(stop_words)

{'t', 'don', 're', "you'll", 'wouldn', 'these', 'here', 'this', 'more', 'her', 'that', 've', 'by', 'above', 'has', 'himself', "won't", 'up', 'are', 'of', 'ma', 'after', 'does', 'needn', 'our', 'down', 'into', 'under', 'isn', 'below', 'themselves', "don't", 'both', 'just', "mightn't", 'over', 'only', "hadn't", 'aren', 'were', 'm', "wasn't", 'weren', 'again', 'why', 'ain', 'yourselves', 'or', 'having', 'my', 'wasn', 'his', 'can', 'hadn', "haven't", 'doing', 'about', 'she', "shan't", 'yourself', 'now', 'their', 'an', "couldn't", 'mightn', 'hers', 'o', 'such', "isn't", 'is', 'same', 'to', "mustn't", 'they', "you'd", 'each', "you've", 'had', 'where', 'against', "that'll", 'from', 'once', 'hasn', "she's", 'few', "doesn't", 'through', 'did', 'too', 'further', 'what', 'but', 'because', 'no', "hasn't", 'with', 'mustn', "weren't", 'other', 'while', "didn't", "shouldn't", "you're", 'very', 'whom', "should've", 'and', 'there', 'not', 'theirs', 'own', 'some', 'me', 'so', 'for', 'it', 'do', "wouldn'

In [208]:
import string
punct=string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [209]:
# loading the dataset to a pandas DataFrame
train = pd.read_csv('file_path', encoding='latin-1')
test= pd.read_csv('file_path', encoding='latin-1')

In [210]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [211]:
test.head()

,ItemID,SentimentText
0,1,is so sad for my APL frie...
1,2,I missed the New Moon trail...
2,3,omg its already 7:30 :O
3,4,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,i think mi bf is cheating on me!!! ...


In [212]:
test.shape

(299989, 2)

In [213]:
train_x=train["SentimentText"]
train_y=train["Sentiment"]

In [214]:
train_x

0                             is so sad for my APL frie...
1                           I missed the New Moon trail...
2                                  omg its already 7:30 :O
3                  .. Omgaga. Im sooo  im gunna CRy. I'...
4                 i think mi bf is cheating on me!!!   ...
                               ...                        
99984    @Cupcake  seems like a repeating problem   hop...
99985    @cupcake__ arrrr we both replied to each other...
99986                       @CuPcAkE_2120 ya i thought so 
99987    @Cupcake_Dollie Yes. Yes. I'm glad you had mor...
99988                      @cupcake_kayla haha yes you do 
Name: SentimentText, Length: 99989, dtype: object

In [215]:
train_y

0        0
1        0
2        1
3        0
4        0
        ..
99984    0
99985    1
99986    0
99987    1
99988    1
Name: Sentiment, Length: 99989, dtype: int64

In [216]:
train_y.value_counts()

1    56457
0    43532
Name: Sentiment, dtype: int64

In [217]:
test_x=test["SentimentText"]

In [218]:
print(test_x)

0                              is so sad for my APL frie...
1                            I missed the New Moon trail...
2                                   omg its already 7:30 :O
3                   .. Omgaga. Im sooo  im gunna CRy. I'...
4                  i think mi bf is cheating on me!!!   ...
                                ...                        
299984    @martine2323 'Morning, Petal - work brought me...
299985    @martinetrene Hello,little bird!  i finally st...
299986    @MartinezMayra I got u at work when I go buy o...
299987       @martinfaux He certainly gives good interview 
299988    @martinfaux we've the feedback about our brows...
Name: SentimentText, Length: 299989, dtype: object


In [219]:
train_x.isnull().sum()

0

In [220]:
test_x.isnull().sum()

0

In [221]:
def clean(text):
    content=nltk.word_tokenize(text.lower())
    content=re.sub('https?://\S+','',str(content))
    content=re.sub('[^a-zA-Z]',' ',str(content))
    content=re.sub(r'\s+',' ',str(content))
    content =content.split()
    lemma=WordNetLemmatizer()
    content = [lemma.lemmatize(word , pos='v') for word in content if (not word in punct) & (not word in stop_words)]
    content = ' '.join(content)
    return content

In [222]:
x_train=train_x.apply(clean)

In [223]:
print(x_train)

0                                           sad apl friend
1                                    miss new moon trailer
2                                              omg already
3        omgaga im sooo im gunna cry dentist since supo...
4                                        think mi bf cheat
                               ...                        
99984    cupcake seem like repeat problem hope able fin...
99985    cupcake arrrr reply different tweet time see d...
99986                                     cupcake ya think
99987                      cupcake dollie yes yes glad fun
99988                               cupcake kayla haha yes
Name: SentimentText, Length: 99989, dtype: object


In [224]:
x_test=test_x.apply(clean)

In [225]:
print(x_test)

0                                            sad apl friend
1                                     miss new moon trailer
2                                               omg already
3         omgaga im sooo im gunna cry dentist since supo...
4                                         think mi bf cheat
                                ...                        
299984    martine morning petal work bring bank first ho...
299985    martinetrene hello little bird finally start r...
299986    martinezmayra get u work go buy one u know ima...
299987             martinfaux certainly give good interview
299988    martinfaux feedback browser compatibility stor...
Name: SentimentText, Length: 299989, dtype: object


In [226]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
train_vec=vectorizer.fit_transform(x_train)

In [227]:
print(train_vec)

  (0, 53037)	0.43289543577431105
  (0, 13966)	0.8213296762766267
  (0, 76990)	0.371509225394338
  (1, 84803)	0.6313626989965042
  (1, 67205)	0.5879779843568281
  (1, 68724)	0.37754282376282255
  (1, 66653)	0.33633978125212244
  (2, 7776)	0.6929494310444987
  (2, 70349)	0.7209861898920848
  (3, 66564)	0.2709961162961676
  (3, 74155)	0.20789100491713808
  (3, 45846)	0.33505128265420164
  (3, 53923)	0.1152570917327998
  (3, 81971)	0.4065813259687981
  (3, 79121)	0.21214316178108042
  (3, 47582)	0.3052965129575028
  (3, 45978)	0.21665036099659432
  (3, 55274)	0.2925777034984362
  (3, 80114)	0.2382306973605294
  (3, 58196)	0.3157190016251374
  (3, 70350)	0.4065813259687981
  (4, 37671)	0.5694773172179313
  (4, 24588)	0.5280535336974084
  (4, 66205)	0.5694773172179313
  (4, 83701)	0.2693522524468943
  :	:
  (99984, 52075)	0.2718610961290102
  (99984, 63246)	0.20059892122407363
  (99984, 80031)	0.2902905175917009
  (99985, 61019)	0.45432530742549154
  (99985, 15392)	0.45432530742549154
  (999

In [228]:
test_vec=vectorizer.transform(x_test)

In [229]:
print(test_vec)

  (0, 76990)	0.371509225394338
  (0, 53037)	0.43289543577431105
  (0, 13966)	0.8213296762766267
  (1, 84803)	0.6313626989965042
  (1, 68724)	0.37754282376282255
  (1, 67205)	0.5879779843568281
  (1, 66653)	0.33633978125212244
  (2, 70349)	0.7209861898920848
  (2, 7776)	0.6929494310444987
  (3, 81971)	0.4065813259687981
  (3, 80114)	0.2382306973605294
  (3, 79121)	0.21214316178108042
  (3, 74155)	0.20789100491713808
  (3, 70350)	0.4065813259687981
  (3, 66564)	0.2709961162961676
  (3, 58196)	0.3157190016251374
  (3, 55274)	0.2925777034984362
  (3, 53923)	0.1152570917327998
  (3, 47582)	0.3052965129575028
  (3, 45978)	0.21665036099659432
  (3, 45846)	0.33505128265420164
  (4, 83701)	0.2693522524468943
  (4, 66205)	0.5694773172179313
  (4, 37671)	0.5694773172179313
  (4, 24588)	0.5280535336974084
  :	:
  (299985, 64170)	0.22790243126200965
  (299985, 63458)	0.2450076759229728
  (299985, 56367)	0.278616915385677
  (299985, 53923)	0.1434788143416147
  (299985, 52065)	0.28583759317142093
  (

In [230]:
# Training the Model: MultinomialNB
model= MultinomialNB()


In [231]:
model.fit(train_vec,train_y)

MultinomialNB()

In [232]:
# accuracy score on the training data
train_prediction = model.predict(train_vec)
training_data_accuracy = accuracy_score(train_prediction, train_y)

In [233]:
print(training_data_accuracy)

0.8593945333986739


In [234]:
test_prediction = model.predict(test_vec)

In [235]:
print(test_prediction)

[0 1 0 ... 0 1 1]


In [236]:
#saving and loading the model
import pickle

In [237]:
pickle.dump(vectorizer, open('tranform.pkl', 'wb'))
pickle.dump(model, open('model.pkl', 'wb'))

In [238]:
vec=pickle.load(open('tranform.pkl', 'rb'))
mp=pickle.load(open('model.pkl', 'rb'))